In [1]:
import pandas as pd
import urllib.request
import urllib.parse
import json
import re, sys
import datetime
from IPython.core.debugger import Tracer; debug_here = Tracer()

In [8]:
# GET URLS
talk_data = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/ipc_talkpages_byarticle.csv')
talk_data['article_rev_url'] = pd.Series()
# talk_data

prev_art = ''

# for i, row in talk_data.loc[8000:8500,:].iterrows():
for i, row in talk_data.iterrows():
    
    if i % 500 == 0:
        print(i/1000, 'k')
        sys.stdout.flush()
    
    if row['article_title'] != prev_art:

        # Build article revision table
        baseurl = "https://en.wikipedia.org/w/api.php?action=query&titles={:s}&prop=revisions&rvprop=ids|timestamp&rvlimit=max&format=json"

        url = baseurl.format(urllib.parse.quote(row['article_title']))

        retrieved =  urllib.request.urlopen(url).read().decode('utf-8')

        pagedict = json.loads(retrieved)

        # page no longer exists with that name
        if not 'revisions' in pagedict['query']['pages'][list(pagedict['query']['pages'].keys())[0]]:
            continue
            
        revtable = pagedict['query']['pages'][list(pagedict['query']['pages'].keys())[0]]['revisions']

        # Loop thru continues
        while 'continue' in pagedict.keys():
            cnum = re.search(r'rvcontinue":"(.*?)"', retrieved).group(1)

            baseurl = "https://en.wikipedia.org/w/api.php?action=query&titles={:s}&prop=revisions&rvprop=ids|timestamp&rvlimit=max&rvcontinue={:s}&format=json"

            url = baseurl.format(urllib.parse.quote(row['article_title']), cnum)

            retrieved =  urllib.request.urlopen(url).read().decode('utf-8')

            pagedict = json.loads(retrieved)

            revtable += pagedict['query']['pages'][list(pagedict['query']['pages'].keys())[0]]['revisions']

        # Parse dates in revision table
        for el in revtable:
            el['timestamp'] = datetime.datetime.strptime(el['timestamp'], '%Y-%m-%dT%H:%M:%SZ')
            
        prev_art = row['article_title']

    # Get most recent article revision before the talk page date
    talk_ts = row['post_timestamp']
    if isinstance(talk_ts, str):
        talk_ts = datetime.datetime.strptime(talk_ts, '%Y-%m-%d %H:%M:%S.0')
        
        # Look for most recent article revision in revtable
        most_recent_revid = ""
        
        for el in reversed(revtable):
            if el['timestamp'] <= talk_ts:
                most_recent_revid = el['revid']
            else:
                break
    
        if most_recent_revid == '': continue # another case where article name changed
            
        rev_url = 'https://en.wikipedia.org/w/index.php?title={0}&oldid={1}'.format(row['article_title'], most_recent_revid)

        talk_data.loc[i, 'article_rev_url'] = rev_url
    
# talk_data.loc[17580:17600,:]

# Print talk_data
talk_data.to_csv('/home/michael/school/research/wp/wikipedia/data/talk/ipc_talk_article_urls.csv', index=False)

talk_data

0.0 k
0.5 k
1.0 k
1.5 k
2.0 k
2.5 k
3.0 k
3.5 k
4.0 k
4.5 k
5.0 k
5.5 k
6.0 k
6.5 k
7.0 k
7.5 k
8.0 k
8.5 k
9.0 k
9.5 k
10.0 k
10.5 k
11.0 k
11.5 k
12.0 k
12.5 k
13.0 k
13.5 k


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>